In [ ]:
import numpy as np
import sklearn as sk
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from  sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def find_outliers_tukey(x):
    q1 = np.percentile(x, 25)
    q3 = np.percentile(x, 75)
    iqr = q3-q1 
    floor = q1 - 1.5*iqr
    ceiling = q3 + 1.5*iqr
    outlier_indices = list(x.index[(x < floor)|(x > ceiling)])
    outlier_values = list(x[outlier_indices])
    return outlier_indices, outlier_values

In [ ]:
def plot_histogram(x):
    plt.hist(x, color='gray', alpha=0.5)
    plt.title("Histogram of '{var_name}'".format(var_name=x.name))
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
def generate_stat(X_stat):
    for names in X_stat.drop(columns=['city','year','weekofyear','week_start_date']):
        X_stat['month_avg_'+names] = X_stat[names].rolling(3 ,min_periods=1).mean()
        X_stat['month_min_'+names] = X_stat[names].rolling(3 ,min_periods=1).min()
        X_stat['month_max_'+names] = X_stat[names].rolling(3 ,min_periods=1).max()
        X_stat['lag_3_'+names] = X_stat[names].shift(4)
        X_stat['byte_'+names] = X_stat[names].shift(1)
    return X_stat


In [ ]:
def find_correlation(data, threshold=0.9):
    corr_mat = data.corr()
    corr_mat.loc[:, :] = np.tril(corr_mat, k=-1)
    already_in = set()
    result = []
    for col in corr_mat:
        perfect_corr = corr_mat[col][abs(corr_mat[col])> threshold].index.tolist()
        if perfect_corr and col not in already_in:
            already_in.update(set(perfect_corr))
            perfect_corr.append(col)
            result.append(perfect_corr)
    select_nested = [f[1:] for f in result]
    select_flat = [i for j in select_nested for i in j]
    return select_flat

In [ ]:
def PP_Y(par_Y):
    par_Y = par_Y[(par_Y['city']=='iq')]
    return par_Y

In [ ]:
def PP_X(par_X):
    par_X['week_start_date'] = pd.to_datetime(par_X['week_start_date'])
#     par_X = par_X[(par_X['city']=='iq')]
    par_X = generate_stat(par_X)
    par_X = par_X.drop(columns=[
        'weekofyear', 
        'week_start_date',
        'year',
        'city',
        
    ])
    par_X = par_X.fillna(method = 'bfill')
    par_X = par_X.fillna(method = 'ffill')
    return par_X

In [ ]:
X = pd.read_csv("dengue_features_train.csv")
Y = pd.read_csv("dengue_labels_train.csv")
X_New = pd.read_csv("dengue_features_test.csv")
Y_New = pd.read_csv('submission_format.csv')
Xrf = X[(X['city'] == 'iq')]
Yrf = Y[(X['city'] == 'iq')]['total_cases'].values.ravel()
Y_New = Y_New[(Y_New['city'] == 'iq')]
X_New = X_New[(X_New['city'] == 'iq')]
X_train, X_test, Y_train, Y_test = Xrf[:int(len(Xrf)*0.8)], Xrf[int(len(Xrf)*0.8):] , Yrf[:int(len(Yrf)*0.8)] , Yrf[int(len(Yrf)*0.8):]
X_train, X_test , X_New = PP_X(X_train),PP_X(X_test) , PP_X(X_New)

In [ ]:
Xrf

In [ ]:
db = KMeans(n_clusters=5, random_state=0)
fit = db.fit(X_train)
fit.predict(X_test)

# core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
# core_samples_mask[db.core_sample_indices_] = True
# labels = db.labels_
# n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
# n_clusters_

## outlier

In [ ]:
X , Y = X.loc[Y['total_cases'] <40] , Y.loc[Y['total_cases'] <40]

In [ ]:
pca = PCA(n_components=10)
X2 = pd.DataFrame(pca.fit_transform(X),index=X.index)
X = pd.concat([X,X2],axis=1)
X_New2 = pd.DataFrame(pca.transform(X_New), index = X_New.index)
X_New = pd.concat([X_New,X_New2],axis=1)

# X = pd.DataFrame(pca.fit_transform(X),index=X.index)
# X_New = pd.concat([X_New,X_New2],axis=1)



## itteratively drop columns with high absalute correlation

In [ ]:
columns_to_drop = find_correlation(X_train , 0.9)
X_test = X_test.drop(columns=columns_to_drop)
X_train = X_train.drop(columns=columns_to_drop)
X_New = X_New.drop(columns=columns_to_drop)

In [ ]:
X_crr = X_train.copy(deep=True)
X_crr['total_cases'] = Y['total_cases']
corr = X_crr.corr()

In [ ]:
linear_features=abs(corr).total_cases.drop('total_cases').sort_values(ascending=False)[:10].keys()

In [ ]:
rf = RandomForestRegressor()
model1 = rf.fit(X_train,Y_train)
predict = rf.predict(X_test)                                              
print(mean_absolute_error(Y_test, predict.round(0).astype(int)))  
Y_rf = model1.predict(X_New)

In [ ]:
plt.figure(figsize=(20,15))
importance = rf.feature_importances_
feat_importances_act = pd.Series(importance, index=X_train.columns)
feat_importances = feat_importances_act.nlargest(20)
feat_importances.plot(kind='barh')

In [ ]:
from sklearn import linear_model

reg = linear_model.Lasso (alpha = 0.0005 , positive=True)
reg.fit(X_train[linear_features],Y_train)
predict2 = reg.predict(X_test[linear_features])    
print(mean_absolute_error(Y_test, [max(x,0) for x in predict2.round(0).astype(int)]))
predict_New2 = reg.predict(X_New[linear_features])
predict_New2= [max(x,0) for x in predict_New2]
predict_New2

In [ ]:
import statsmodels.api as sm
model_nb = sm.GLM(Y_train,X_train, family=sm.families.NegativeBinomial()).fit()
predict2 = model_nb.predict(X_test)
print(mean_absolute_error(Y_test, [max(x,0) for x in predict2.round(0).astype(int)]))
predict_New2 = model_nb.predict(X_New)
predict_New2

In [ ]:
model1.predict(X_testrf)

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train[feat_importances_act.nlargest(10).keys()],Y_train)
predict3 = clf.predict(X_test[feat_importances_act.nlargest(10).keys()])     
print(mean_absolute_error(Y_test, predict3.round(0).astype(int)))  
clf.predict(X_testrf[feat_importances_act.nlargest(10).keys()])


In [ ]:
from statsmodels.tsa.arima_model import ARIMA



In [ ]:
Y_New['total_cases'] = np.array(predict_New2, dtype=np.float32).round(0)
Y_New['total_cases'] = Y_New['total_cases'].astype(int)
Y_New.to_csv('submission_iq.csv')
Y_New

In [ ]:
print(mean_absolute_error(Y_test, predict2.round(0).astype(int)))  
predict_avg = (predict+predict2)/2
print(mean_absolute_error(Y_test, predict_avg.round(0).astype(int)))  


In [ ]:
plt.figure(figsize=(20,15))
abs(corr).total_cases.drop('total_cases').sort_values(ascending=False)[:10].plot.barh()

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)